# Optimization of an agent in Flappy Bird game using Genetic Algorithm

In this project we will develop an agent that plays Flappy Bird using Neural Networks and Genetic Algorithm

# Basic Imports

In [1]:
import pygame
import random
import numpy as np
import torch
import torch.nn as nn

pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Constants

In [2]:
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
PIPE_SPEED = 4   # How many pixels the pipe moves to the left each frame
GRAVITY = 0.5    # How much the bird's vertical speed increases downward in each frame
BIRD_JUMP = -8
POPULATION_SIZE = 50  # How many birds (agents) do we test simultaneously in one round (generation)
MUTATION_RATE = 0.1   # Strength of "gene"(weights) change. If it is 0.1, we change the weights by approximately 10%.

# Neural Network

Inputs(5) : y bird position, y top pipe, y lower pipe, horizontal distance, bird velocity

Hidden layer : 16 nodes

Output layer : 1 node

In [3]:
class BirdBrain(nn.Module):
    def __init__(self):
        super(BirdBrain, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(5, 16),  #5 inputs, hidden layer has 16 nodes
            nn.ReLU(),  #activation, if the signal is negative, set it to zero
            nn.Linear(16, 1),  #from hidden layer to output layer that determines wether the bird should jump
            nn.Sigmoid(),    #if output is >0.5, JUMP
        )

        for param in self.parameters():  # turning off "gradient learning"
            param.requires_grad = False

    def forward(self, x):
        return self.fc(x)

# Game Logic

**Agent**:

In [4]:
class Bird:
    def __init__(self):
        self.y = SCREEN_HEIGHT // 2    #y position of the bird
        #normalized (0-1) relative to the screen height


        self.vel = 0
        self.brain = BirdBrain()

        #the longer the bird is alive in the game, the better the fitness
        #GA will use this number to see who is the 'winner'
        #crucial for evolution
        self.fitness = 0 

        self.alive = True 
        self.score = 0

    def jump(self):
        self.vel = BIRD_JUMP

    def update(self, pipe_info):
        '''
            The bird looks at the pipe_info, sends the information to the brain.
            gets an answer (0-1) and decides whether it should jump
        '''

        if not self.alive : return


        '''
            We dont increment fitness for passing through the pipes.
            Because at the beginning all the birds would have 0 points (because none of them know how to pass the pipe). 
            In this way, the one who "survived" longer, even if only by falling, gets a slightly higher grade, which directs the evolution.
        '''
        self.fitness += 0.1 #reward for surviving the frame

        self.vel += GRAVITY # in each frame we add the gravity to the velocity and the velocity to the position
        self.y += self.vel  # this creates a realistic feeling of "falling"

        # DECISION PROCESS - AI
        inputs = torch.tensor(
            [
                self.y / SCREEN_HEIGHT,
                pipe_info['top'] / SCREEN_HEIGHT,
                pipe_info['bottom'] / SCREEN_HEIGHT,
                pipe_info['dist'] / SCREEN_WIDTH,
                self.vel / 15
            ],
            dtype=torch.float32
        )   # normalization (0-1)


        # check if the bird should jump
        if self.brain(inputs).item() > 0.5:
            self.jump()


        # check for collision
        if self.y < 0 or self.y > SCREEN_HEIGHT:
            self.alive = False



In [5]:
class Pipe:
    def __init__(self, x):
        self.x =x
        self.gap = 150
        self.top = random.randint(50, SCREEN_HEIGHT - self.gap - 50)
        self.bottom = self.top + self.gap
        self.passed = False
    
    def update(self):
        self.x -= PIPE_SPEED

# Genetic Algorithm

In [6]:
def mutate(model):
    for param in model.parameters():   #going through all the weights
        mutation = torch.randn_like(param) * MUTATION_RATE    # generates random numbers of the same size as the weights
        param.add_(mutation)     # adding that to existing weights. giving the bird a little "change of mind"

In [7]:
def crossover(parent1, parent2, child):
    for p1, p2, c in zip(parent1.parameters(), parent2.parameters(), child.parameters()):
        mask = torch.rand_like(p1) > 0.5          #  create a random map of zeros and ones (50/50 chance)
        c.data = torch.where(mask, p1.data, p2.data) 
        # Where the mask is 1, take the gene from parent 1; where 0 is, take it from parent 2. The result goes to the child

# Game Loop

In [8]:
def main():
    pygame.init()    #start all pygame modules
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Arial", 24)

    population = [Bird() for _ in range(POPULATION_SIZE)]  #starting with a population of 50 birds that all have their own brain
    generation = 1   # counter that tells us how many times we've tried to evolve birds so far

    while True :   #going through generations
        pipes = [Pipe(SCREEN_WIDTH + 100)]
        running = True

        while running:   #continues as long as there are live birds on the screen
            # each loop is one frame
            screen.fill((135, 206, 235))   #sky blue

            if pipes[-1].x < SCREEN_WIDTH - 250:
                pipes.append(Pipe(SCREEN_WIDTH))
            if pipes[0].x < -50:
                pipes.pop(0)



            '''
                The bird needs to analyze only the next pipe (target pipe)
                If the bird has passed a pipe, the next one becomes the target pipe
            '''
            target_pipe = pipes[0]
            if target_pipe.x + 50 < 50: 
                target_pipe = pipes[1]

            pipe_info = {'top': target_pipe.top, 'bottom': target_pipe.bottom, 'dist': target_pipe.x}



            all_dead = True
            for bird in population:
                if bird.alive:
                    all_dead = False
                    bird.update(pipe_info)
                    # collision
                    if target_pipe.x < 100 and target_pipe.x + 50 > 50:
                        if bird.y < target_pipe.top or bird.y > target_pipe.bottom:
                            bird.alive = False
                    
                    # drawing the bird
                    pygame.draw.circle(screen, (255, 255, 0), (50, int(bird.y)), 15)

            if all_dead:  
                running = False  #exit inner loop

            # draw pipes
            for pipe in pipes:
                pipe.update()
                pygame.draw.rect(screen, (0, 255, 0), (pipe.x, 0, 50, pipe.top))
                pygame.draw.rect(screen, (0, 255, 0), (pipe.x, pipe.bottom, 50, SCREEN_HEIGHT))

            
            img = font.render(f"Generation: {generation}  Alive: {sum(b.alive for b in population)}", True, (0,0,0))
            screen.blit(img, (10, 10))

            pygame.display.flip()
            clock.tick(60)
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit(); return

        # evolution, after all birds are dead
        population.sort(key=lambda x: x.fitness, reverse=True)
        best_birds = population[:5] # top 5 become parents
        
        new_population = []
        # elitism-keeping the best birds
        for i in range(2):
            elite = Bird()
            elite.brain.load_state_dict(best_birds[i].brain.state_dict())
            new_population.append(elite)
            
        # making the children
        while len(new_population) < POPULATION_SIZE:
            p1, p2 = random.sample(best_birds, 2)  
            child = Bird()
            crossover(p1.brain, p2.brain, child.brain)
            mutate(child.brain)
            new_population.append(child)
            
        population = new_population
        generation += 1



if __name__ == "__main__":
    main()